In [38]:
import pandas as pd
import calendar
import numpy as np
from datetime import datetime

pd.set_option('display.max_columns', None)

In [39]:
df_flights = pd.read_csv('./flight-delays/flights.csv')
df_airlines = pd.read_csv('./flight-delays/airlines.csv')
df_airports = pd.read_csv('./flight-delays/airports.csv')

- **Flights Dataset**

In [3]:
df_flights.head()

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,0,363169,5290975,2015,11,28,6,DL,1590,N971AT,CAE,ATL,700,655.0,-5.0,17.0,712.0,68.0,61.0,39.0,192,751.0,5.0,808,756.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,704061,4899614,2015,11,2,1,AA,2516,N3CVAA,ORD,DEN,1715,1743.0,28.0,40.0,1823.0,164.0,177.0,126.0,888,1929.0,11.0,1859,1940.0,41.0,0,0,NaN,13.0,0.0,28.0,0.0,0.0
2,2,389056,3769408,2015,8,22,6,AS,93,N317AS,SEA,ANC,1355,1353.0,-2.0,20.0,1413.0,220.0,206.0,182.0,1448,1615.0,4.0,1635,1619.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,132167,4999624,2015,11,9,1,AA,2383,N871AA,MCO,DFW,650,652.0,2.0,16.0,708.0,181.0,172.0,142.0,985,830.0,14.0,851,844.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,304371,2572568,2015,6,12,5,US,1978,N833AW,CLE,CLT,705,700.0,-5.0,12.0,712.0,93.0,92.0,71.0,430,823.0,9.0,838,832.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_flights.drop(columns=['Unnamed: 0.2', 'DAY_OF_WEEK'], inplace=True)
df_flights.rename(columns={'Unnamed: 0': 'F1'}, inplace=True)
df_flights.rename(columns={'Unnamed: 0.1': 'F2'}, inplace=True)
df_flights.rename(columns= lambda x: x.lower(), inplace=True)

df_flights.head()

,f1,f2,year,month,day,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,363169,5290975,2015,11,28,DL,1590,N971AT,CAE,ATL,700,655.0,-5.0,17.0,712.0,68.0,61.0,39.0,192,751.0,5.0,808,756.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,704061,4899614,2015,11,2,AA,2516,N3CVAA,ORD,DEN,1715,1743.0,28.0,40.0,1823.0,164.0,177.0,126.0,888,1929.0,11.0,1859,1940.0,41.0,0,0,NaN,13.0,0.0,28.0,0.0,0.0
2,389056,3769408,2015,8,22,AS,93,N317AS,SEA,ANC,1355,1353.0,-2.0,20.0,1413.0,220.0,206.0,182.0,1448,1615.0,4.0,1635,1619.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,132167,4999624,2015,11,9,AA,2383,N871AA,MCO,DFW,650,652.0,2.0,16.0,708.0,181.0,172.0,142.0,985,830.0,14.0,851,844.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,304371,2572568,2015,6,12,US,1978,N833AW,CLE,CLT,705,700.0,-5.0,12.0,712.0,93.0,92.0,71.0,430,823.0,9.0,838,832.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_flights['cancellation_reason'].value_counts()

B    2397
A    1260
C     776
Name: cancellation_reason, dtype: int64

In [6]:
df_flights.loc[df_flights['cancellation_reason'] == 'A', 'cancellation_reason'] = 'Airline/Carrier'
df_flights.loc[df_flights['cancellation_reason'] == 'B', 'cancellation_reason'] = 'Weather'
df_flights.loc[df_flights['cancellation_reason'] == 'C', 'cancellation_reason'] = 'National Air System'

In [7]:
df_flights['cancellation_reason'].value_counts()

Weather                2397
Airline/Carrier        1260
National Air System     776
Name: cancellation_reason, dtype: int64

In [8]:
df_flights.head()

,f1,f2,year,month,day,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,363169,5290975,2015,11,28,DL,1590,N971AT,CAE,ATL,700,655.0,-5.0,17.0,712.0,68.0,61.0,39.0,192,751.0,5.0,808,756.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,704061,4899614,2015,11,2,AA,2516,N3CVAA,ORD,DEN,1715,1743.0,28.0,40.0,1823.0,164.0,177.0,126.0,888,1929.0,11.0,1859,1940.0,41.0,0,0,NaN,13.0,0.0,28.0,0.0,0.0
2,389056,3769408,2015,8,22,AS,93,N317AS,SEA,ANC,1355,1353.0,-2.0,20.0,1413.0,220.0,206.0,182.0,1448,1615.0,4.0,1635,1619.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,132167,4999624,2015,11,9,AA,2383,N871AA,MCO,DFW,650,652.0,2.0,16.0,708.0,181.0,172.0,142.0,985,830.0,14.0,851,844.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,304371,2572568,2015,6,12,US,1978,N833AW,CLE,CLT,705,700.0,-5.0,12.0,712.0,93.0,92.0,71.0,430,823.0,9.0,838,832.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_flights.insert(loc=5, column='date_of_flight', value=df_flights[['year', 'month', 'day']].astype(str).apply(lambda x: '-'.join(x), axis=1))
df_flights.head()

,f1,f2,year,month,day,date_of_flight,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,363169,5290975,2015,11,28,2015-11-28,DL,1590,N971AT,CAE,ATL,700,655.0,-5.0,17.0,712.0,68.0,61.0,39.0,192,751.0,5.0,808,756.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,704061,4899614,2015,11,2,2015-11-2,AA,2516,N3CVAA,ORD,DEN,1715,1743.0,28.0,40.0,1823.0,164.0,177.0,126.0,888,1929.0,11.0,1859,1940.0,41.0,0,0,NaN,13.0,0.0,28.0,0.0,0.0
2,389056,3769408,2015,8,22,2015-8-22,AS,93,N317AS,SEA,ANC,1355,1353.0,-2.0,20.0,1413.0,220.0,206.0,182.0,1448,1615.0,4.0,1635,1619.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,132167,4999624,2015,11,9,2015-11-9,AA,2383,N871AA,MCO,DFW,650,652.0,2.0,16.0,708.0,181.0,172.0,142.0,985,830.0,14.0,851,844.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,304371,2572568,2015,6,12,2015-6-12,US,1978,N833AW,CLE,CLT,705,700.0,-5.0,12.0,712.0,93.0,92.0,71.0,430,823.0,9.0,838,832.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_flights.drop(columns=['year', 'month', 'day'], inplace=True)
df_flights.head()

,f1,f2,date_of_flight,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,363169,5290975,2015-11-28,DL,1590,N971AT,CAE,ATL,700,655.0,-5.0,17.0,712.0,68.0,61.0,39.0,192,751.0,5.0,808,756.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,704061,4899614,2015-11-2,AA,2516,N3CVAA,ORD,DEN,1715,1743.0,28.0,40.0,1823.0,164.0,177.0,126.0,888,1929.0,11.0,1859,1940.0,41.0,0,0,NaN,13.0,0.0,28.0,0.0,0.0
2,389056,3769408,2015-8-22,AS,93,N317AS,SEA,ANC,1355,1353.0,-2.0,20.0,1413.0,220.0,206.0,182.0,1448,1615.0,4.0,1635,1619.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,132167,4999624,2015-11-9,AA,2383,N871AA,MCO,DFW,650,652.0,2.0,16.0,708.0,181.0,172.0,142.0,985,830.0,14.0,851,844.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,304371,2572568,2015-6-12,US,1978,N833AW,CLE,CLT,705,700.0,-5.0,12.0,712.0,93.0,92.0,71.0,430,823.0,9.0,838,832.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_flights.dtypes

f1                       int64
f2                       int64
date_of_flight          object
airline                 object
flight_number            int64
tail_number             object
origin_airport          object
destination_airport     object
scheduled_departure      int64
departure_time         float64
departure_delay        float64
taxi_out               float64
wheels_off             float64
scheduled_time         float64
elapsed_time           float64
air_time               float64
distance                 int64
wheels_on              float64
taxi_in                float64
scheduled_arrival        int64
arrival_time           float64
arrival_delay          float64
diverted                 int64
cancelled                int64
cancellation_reason     object
air_system_delay       float64
security_delay         float64
airline_delay          float64
late_aircraft_delay    float64
weather_delay          float64
dtype: object

In [12]:
df_flights['date_of_flight'] = pd.to_datetime(df_flights['date_of_flight'])
df_flights.head()

,f1,f2,date_of_flight,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,363169,5290975,2015-11-28,DL,1590,N971AT,CAE,ATL,700,655.0,-5.0,17.0,712.0,68.0,61.0,39.0,192,751.0,5.0,808,756.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,704061,4899614,2015-11-02,AA,2516,N3CVAA,ORD,DEN,1715,1743.0,28.0,40.0,1823.0,164.0,177.0,126.0,888,1929.0,11.0,1859,1940.0,41.0,0,0,NaN,13.0,0.0,28.0,0.0,0.0
2,389056,3769408,2015-08-22,AS,93,N317AS,SEA,ANC,1355,1353.0,-2.0,20.0,1413.0,220.0,206.0,182.0,1448,1615.0,4.0,1635,1619.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,132167,4999624,2015-11-09,AA,2383,N871AA,MCO,DFW,650,652.0,2.0,16.0,708.0,181.0,172.0,142.0,985,830.0,14.0,851,844.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,304371,2572568,2015-06-12,US,1978,N833AW,CLE,CLT,705,700.0,-5.0,12.0,712.0,93.0,92.0,71.0,430,823.0,9.0,838,832.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_flights.to_csv('./updated_data/flights.csv', index=False)

- **Airlines Dataset**

In [15]:
df_airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [16]:
df_airlines.rename(columns= lambda x: x.lower(), inplace=True)
df_airlines.head()

,iata_code,airline
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [20]:
df_airlines['iata_code'].value_counts()

UA    1
AA    1
US    1
F9    1
B6    1
OO    1
AS    1
NK    1
WN    1
DL    1
EV    1
HA    1
MQ    1
VX    1
Name: iata_code, dtype: int64

In [17]:
df_airlines.to_csv('./updated_data/airlines.csv', index=False)

- **Airports Dataset**

In [18]:
df_airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [19]:
df_airports.rename(columns= lambda x: x.lower(), inplace=True)
df_airports.head()

,iata_code,airport,city,state,country,latitude,longitude
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [22]:
df_airports.to_csv('./updated_data/airports.csv', index=False)

In [3]:
df = pd.read_csv('./updated_data/flights.csv')
df.head()

,f1,f2,date_of_flight,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,363169,5290975,2015-11-28,DL,1590,N971AT,CAE,ATL,700,655.0,-5.0,17.0,712.0,68.0,61.0,39.0,192,751.0,5.0,808,756.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,704061,4899614,2015-11-02,AA,2516,N3CVAA,ORD,DEN,1715,1743.0,28.0,40.0,1823.0,164.0,177.0,126.0,888,1929.0,11.0,1859,1940.0,41.0,0,0,NaN,13.0,0.0,28.0,0.0,0.0
2,389056,3769408,2015-08-22,AS,93,N317AS,SEA,ANC,1355,1353.0,-2.0,20.0,1413.0,220.0,206.0,182.0,1448,1615.0,4.0,1635,1619.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,132167,4999624,2015-11-09,AA,2383,N871AA,MCO,DFW,650,652.0,2.0,16.0,708.0,181.0,172.0,142.0,985,830.0,14.0,851,844.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,304371,2572568,2015-06-12,US,1978,N833AW,CLE,CLT,705,700.0,-5.0,12.0,712.0,93.0,92.0,71.0,430,823.0,9.0,838,832.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
